<a href="https://colab.research.google.com/github/Moisito12/Tensorflow-2.0/blob/master/Aprendizaje_por_Transferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Paso 1: Instalación de dependencias y configuración del entorno en GPU
**

In [0]:
!pip install tensorflow

In [0]:
!pip install tqdm

**Descargar el dataset de Perros vs Gatos
**

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-12-19 22:04:47--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   208MB/s    in 0.3s    

2019-12-19 22:04:53 (208 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



*Paso 2: Pre procesado del data set*

Importar las dependencias del proyecto

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'1.15.0'

**Descomprimir el dataset de perros y gatos**

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall('./')

In [0]:
zip_object.close()

**Configurar las rutas al dataset**

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

**Construir el Modelo**

**Cargar un modelo pre entrenado (MobileNetV2)**

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights="imagenet")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
9412608/9406464 [==============================] - 1s 0us/step


In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

**Congelar el modelo base**

In [0]:
base_model.trainable = False

**Definir la cabecera personalizada para nuestra red neuronal**

In [0]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(?, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(?, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

**Definir el modelo**

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

**Compilar el modelo**

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.00001),loss="binary_crossentropy",metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**Crear generadores de datos**

**Redimensionar imágenes** = Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.

Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).


In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_validate = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128),batch_size=128,class_mode="binary")

Found 2000 images belonging to 2 classes.


In [0]:
valida_generator = data_gen_validate.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128,class_mode="binary")

Found 1000 images belonging to 2 classes.


**Entrenar el modelo**

In [0]:
model.fit_generator(train_generator,epochs=8,validation_data=valida_generator)

Epoch 1/8
16/16 [==============================] - 97s 6s/step - loss: 0.6225 - acc: 0.6420 - val_loss: 0.6478 - val_acc: 0.6350
Epoch 2/8
16/16 [==============================] - 93s 6s/step - loss: 0.6183 - acc: 0.6565 - val_loss: 0.6395 - val_acc: 0.6430
Epoch 3/8
16/16 [==============================] - 94s 6s/step - loss: 0.6188 - acc: 0.6700 - val_loss: 0.6333 - val_acc: 0.6450
Epoch 4/8
16/16 [==============================] - 94s 6s/step - loss: 0.6065 - acc: 0.6665 - val_loss: 0.6257 - val_acc: 0.6450
Epoch 5/8
16/16 [==============================] - 94s 6s/step - loss: 0.6004 - acc: 0.6760 - val_loss: 0.6194 - val_acc: 0.6470
Epoch 6/8
16/16 [==============================] - 93s 6s/step - loss: 0.5919 - acc: 0.6785 - val_loss: 0.6127 - val_acc: 0.6630
Epoch 7/8
16/16 [==============================] - 93s 6s/step - loss: 0.5878 - acc: 0.6825 - val_loss: 0.6051 - val_acc: 0.6690
Epoch 8/8
16/16 [==============================] - 93s 6s/step - loss: 0.5856 - acc: 0.6810 - val

**Evaluar el modelo de aprendizaje por transferencia**

In [0]:
valid_loss,valid_accuracy = model.evaluate_generator(valida_generator)

In [0]:
print("Accuracy after transfer learning {}".format(valid_accuracy))

Accuracy after transfer learning 0.6740000247955322


**Puesta a punto de parámetros**

**Un par de cosas:**

° NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.


° Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

**Descongelar unas cuantas capas superiores del modelo**

In [0]:
base_model.trainable = True

In [0]:
print("The number of layers in the base model is {}".format(len(base_model.layers)))

The number of layers in the base model is 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

**Compilar el modelo para la puesta a punto**

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

**Puesta a punto**

In [0]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valida_generator)

Epoch 1/5
16/16 [==============================] - 128s 8s/step - loss: 0.2023 - acc: 0.9165 - val_loss: 0.1180 - val_acc: 0.9600
Epoch 2/5
16/16 [==============================] - 120s 7s/step - loss: 0.0342 - acc: 0.9955 - val_loss: 0.1293 - val_acc: 0.9610
Epoch 3/5
16/16 [==============================] - 119s 7s/step - loss: 0.0150 - acc: 0.9990 - val_loss: 0.1758 - val_acc: 0.9500
Epoch 4/5
16/16 [==============================] - 120s 7s/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.1507 - val_acc: 0.9620
Epoch 5/5
16/16 [==============================] - 119s 7s/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.1145 - val_acc: 0.9670
